In [12]:
from random import randint

import numpy as np
import pandas as pd
from fastai.tabular.all import *

First import pandas to help us work with the collision data excel file.

In [13]:
pd.set_option('display.max_columns', None)
raw_collisions = pd.read_csv("/home/alec/Desktop/code/personal_projects/safe-path-finder/data/raw_collisions.csv", low_memory=False)
raw_collisions["accident"] = [1] * 242131
add_datepart(raw_collisions, 'INCDATE')


/home/alec/Desktop/code/personal_projects/safe-path-finder/.venv/lib/python3.10/site-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)


,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INJURIES,SERIOUSINJURIES,FATALITIES,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR,SOURCE,SOURCEDESC,ADDBY,ADDDTTM,MODBY,MODDTTM,accident,INCDATEYear,INCDATEMonth,INCDATEWeek,INCDATEDay,INCDATEDayofweek,INCDATEDayofyear,INCDATEIs_month_end,INCDATEIs_month_start,INCDATEIs_quarter_end,INCDATEIs_quarter_start,INCDATEIs_year_end,INCDATEIs_year_start,INCDATEElapsed
0,-122.311548,47.607109,1,18100,18100,1799155,Unmatched,Block,NaN,16TH AVE BETWEEN E JEFFERSON ST AND E CHERRY ST,NaN,NaN,0,Unknown,NaN,0,0,0,0,0,0,0,10/7/2004,Mid-Block (not related to intersection),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END AT ANGLE",NaN,NaN,NaN,NaN,NaN,NaN,4281024.0,NaN,NaN,NaN,0,0,N,PTCR,Police Traffic Collision Report,ECKLEYS,2004/10/21 00:00:00+00,ECKLEYS,2004/10/21 00:00:00+00,1,2004,10,41,7,3,281,False,False,False,False,False,False,1.097107e+09
1,-122.344604,47.697445,2,19900,19900,1079890,Unmatched,Block,NaN,AURORA AVE N BETWEEN N 94TH ST AND N 95TH ST,NaN,NaN,0,Unknown,NaN,0,0,0,0,0,0,0,9/23/2004,Mid-Block (but intersection related),16.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE SIDESWIPE",NaN,NaN,NaN,NaN,NaN,NaN,4267039.0,NaN,NaN,NaN,0,0,N,PTCR,Police Traffic Collision Report,ECKLEYS,2004/10/13 00:00:00+00,ECKLEYS,2004/10/13 00:00:00+00,1,2004,9,39,23,3,267,False,False,False,False,False,False,1.095898e+09
2,-122.288424,47.564962,3,101200,101200,3296458,Matched,Block,NaN,RAINIER AVE S BETWEEN S ADAMS ST AND S GENESEE N ST,NaN,NaN,2,Injury Collision,Rear Ended,2,0,0,2,2,0,0,6/14/2009 5:20:00 AM,Mid-Block (but intersection related),14.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Dawn,NaN,9165016.0,Y,14.0,From same direction - both going straight - one stopped - rear-end,0,0,N,PTCR,Police Traffic Collision Report,GIVENSA,2009/06/16 00:00:00+00,STATE INTERFACE,2009/11/12 00:00:00+00,1,2009,6,24,14,6,165,False,False,False,False,False,False,1.244938e+09
3,NaN,NaN,4,53800,53800,2609018,Matched,Block,NaN,BROAD ST BETWEEN HARRISON ST AND BROAD ST EB ON RP,NaN,NaN,2,Injury Collision,Left Turn,2,0,0,2,2,0,0,10/4/2006,Mid-Block (but intersection related),16.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE SIDESWIPE",NaN,0,Clear,Dry,Daylight,NaN,6277029.0,NaN,15.0,From same direction - one left turn - one straight,0,0,N,PTCR,Police Traffic Collision Report,ECKLEYS,2006/10/30 00:00:00+00,STATE INTERFACE,2007/05/15 00:00:00+00,1,2006,10,40,4,2,277,False,False,False,False,False,False,1.159920e+09
4,-122.289571,47.600070,5,65000,65000,2620883,Matched,Block,NaN,S FRINK PL BETWEEN S WASHINGTON ST AND LAKE WASHINGTON BLVD S,NaN,NaN,1,Property Damage Only Collision,Other,1,0,0,1,0,0,0,3/24/2007 5:34:00 PM,Mid-Block (not related to intersection),26.0,MOTOR VEHICLE STRUCK OBJECT IN ROAD,NaN,0,Raining,Wet,Daylight,NaN,7083003.0,NaN,50.0,Fixed object,0,0,N,PTCR,Police Traffic Collision Report,LUKEL,2007/03/28 00:00:00+00,STATE INTERFACE,2007/05/16 00:00:00+00,1,2007,3,12,24,5,83,False,False,False,False,False,False,1.174694e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242126,-122.368227,47.662569,242127,360959,362439,ED51232,Unmatched,Block,NaN,LEARY WAY NW BETWEEN NW 46TH ST AND NW BALLARD WAY,NaN,NaN,1,Property Damage Only Collision,NaN,2,0,0,0,0,0,0,4/14/2023,Mid-Block (but intersection related),11.0,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END AT ANGLE",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,N,PTCR,Police Traffic Collision Report,BRUKTAH,2023/04/26 00:00:00+00,BRUKTAH,2023/04/26 00:00:00+00,1,2023,4,15,14,4

Here we read our collisions data file into the program as a DataFrame. We set the pandas options to display the max columns so pandas doesn't hide any of the columns when we display the DataFrame. The low memory, False key value pair also ensures the displayed DataFrame doesn't hide any data. Additionally, we add a column for our dependant variable, accident, whether or not there was an accident. Some important columns to look at for the machine learning model are location: ('X', 'Y'), time: 'INCDATE', weather: 'WEATHER', and light conditions: 'LIGHTCOND'. Finally, we'll use the fastai function add_datepart to split up our date column to change the data from categorical to numerical. 

In [14]:
collisions = raw_collisions[[
    'X', 'Y', 'SEVERITYCODE', 'WEATHER', 'LIGHTCOND',
    'INCDATEYear', 'INCDATEMonth', 'INCDATEWeek',
    'INCDATEDay', 'INCDATEDayofweek', 'INCDATEDayofyear'
]].copy()


As this is the first iteration of the machine learning model I will drop columns are not significant factors. Note that it's difficult to decide whether or not a factor is significant. However, for simplicity the columns I will keep are: the location in coordinates, the date columns, the weather, and the light conditions.

In [17]:

dep_var = 'SEVERITYCODE'
# If i want to use root mean squared log error (RMSLE) as the metric for
# the dependant variable than I'll need to take the log of the column
continuous, categorical = cont_cat_split(collisions)
procs = (Categorify, FillMissing)
tp_collisions = TabularPandas(collisions, procs, categorical, continuous, y_names=dep_var)
tp_collisions['SEVERITYCODE'] += 1
collisions = tp_collisions.items
# tp_collisions.items.to_csv("/home/alec/Desktop/code/personal_projects/safe-path-finder/data/collisions.csv")


# I'll possibly need to define splits later (training set and validation set) Probably random splits

Categorify and FillMissing are TabularProcs. These transform data in place by replacing categorical data with numerical data and replacing missing values with the median of the column respectively. Fill missing also adds a boolean column to indicate replaced rows. I add one to all of the data points in the severity code column as 0 will indicate no accident once I generate the fake data. Finally I save this data to a file in the last line of code that is commented out.

In [12]:
new_collisions = pd.DataFrame(columns=collisions.columns)
num = 1
while num <= 242131 * 3:
    if num % 1000 == 0:
        print(num)
        new_collisions.to_csv("/home/alec/Desktop/code/personal_projects/safe-path-finder/data/new_collisions.csv")
        #print(new_collisions.head(10))
    rand_index_1 = randint(0, len(collisions) - 1)
    row = collisions.iloc[rand_index_1].copy(deep=True)
    rand_index_2 = randint(0, len(collisions) - 1)
    rand_x, rand_y = collisions.iloc[rand_index_2]['X'], collisions.iloc[rand_index_2]['Y']
    row['X'], row['Y'] = rand_x, rand_y
    row['SEVERITYCODE'] = 0
    exists = (
        (collisions['X'] == row['X']) &
        (collisions['Y'] == row['Y']) &
        (collisions['INCDATEYear'] == row['INCDATEYear']) &
        (collisions['INCDATEMonth'] == row['INCDATEMonth']) &
        (collisions['INCDATEWeek'] == row['INCDATEWeek']) &
        (collisions['INCDATEDay'] == row['INCDATEDay']) &
        (collisions['INCDATEDayofweek'] == row['INCDATEDayofweek']) &
        (collisions['INCDATEDayofyear'] == row['INCDATEDayofyear'])
    ).any()
    if exists:
        continue
    new_collisions = pd.concat([new_collisions, row.to_frame().T], ignore_index=True)
    num += 1
    # randomly pick a road segment (since the weather/road conditions will be the same
    # for that day/time)
    # check if there was an accident for this road segment at this time
    # if not add the data to the set
    # make sure to change severity code to 0


hello


KeyboardInterrupt: 

This code is for generating random data
The algorithm inspired from a description of generating data in this article https://medium.com/geoai/using-machine-learning-to-predict-car-accident-risk-4d92c91a7d57

- Pick random data row, using python random numbers in the range of rows and then accessing that row
- Create a copy of the row and randomly pick an attribute of the row to modify: the intersection, the date, the time of day
- Check if the newly generated data is in the records
- If not add to the dataset
- Loop this code ~4 x the # of records in the current data set.
